# Scikit-Learn의 문서 전처리

## BOW (Bag of Words)
- 문서를 숫자 벡터로 변환하는 가장 기본적인 방법
- 전체 문서(코커스=말뭉치) {d1, d2, ..., dn}를 구성하는 고정된 단어장(vocabulary) {t1, t2, ..., tm} 를 만들고, di라는 개별 문서에 단어장의 단어들이 포함되어 있는지를 표시하는 방법.
- x(i,j) = 문서 di 내의 단어 tj의 출현 빈도
- 또는 x(i,j) = 문서 di 내에 단어 tj가 있으면 1, 없으면 0

## Vectorizer
- 문서를 벡터로 바꾸는 전처리용 클래스
- DictVectorizer: 각 단어 수를 세어놓은 사전에서 BOW 벡터를 만듦
- CountVectorizer: 문서 집합에서 단어 토큰을 생성하고, 각 단어의 수를 세어 BOW 인코딩한 벡터를 만듦
- TfidfVectorizer: CountVectorizer와 유사. but, TF_IDF 방식으로 단어 가중치를 조정
- HashingVectorizer: 해시 함수(hash function)을 사용하여 적은 메모리 사용. 빠른 속도로 BOW 벡터를 만듦.

BOW인코딩 결과는 Sparse 행렬로 만들어지므로, toarray 메서드로 보통 행렬로 변환해야함.
(저장 시 0을 무시하고 저장하는 방법)

### DictVectorizer
- 가장 간단한 형태
- 각 문서에 대해 워드를 잘라 토큰화를 하고, 몇번나왔는지를 세는것이 불편.

In [ ]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
D = [{'A':1, 'B':2}, {'B':3, 'C':1}] #어떤 단어가 몇번 나왔는지를 수동으로 세서 DIctionary로 만들어 놓아야 함.
#첫번째 문서에는 A라는 단어가 1번, B라는 단어가 2번 쓰였고, 두번째 문서에는 B가 3번, C가 1번 쓰였다.
X = v.fit_transform(D)
X

In [ ]:
v.feature_names_  #vocabulary

In [ ]:
v.transform({'C':4, 'D':3})
#D가 들어가면, D는 없는 것으로 처리가 된다.

### CountVectorizer
- 문서를 토큰 리스트로 변환하고, 토큰의 출현 빈도를 세는 작업까지 알아서 해줌.
- 문서를 넣으면 BOW 인코딩 벡터로 변환
- 문서를 넣어주기만 하면 됨.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#Corpus에 리스트로 문서들을 넣어준다.
corpus = [
    'This is the first document.',
    'This is the second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?'
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_ #단어장. 단어와 index number를 dict로

In [ ]:
vect.transform(['This is the second document.']).toarray()

In [ ]:
#Stop_words: 특정 단어 무시

#리스트로 넣어주기
vect = CountVectorizer(stop_words=["and", "is", "the", "this"]).fit(corpus)
print(vect.vocabulary_)

#미리 정해진 stopwords 세트 이용 가능. english = 영어에서 잘 이용되는 stopwords들
vect = CountVectorizer(stop_words="english").fit(corpus)
print(vect.vocabulary_)

In [ ]:
#토큰생성기 선택: analyzer, tokenizer, token_pattern 등의 인수 이용

#analyzer: 캐릭터단위로 선택 가능
vect = CountVectorizer(analyzer="char").fit(corpus)
vect.vocabulary_

In [ ]:
#token_pattern: 정규표현식 이용 가능
vect = CountVectorizer(token_pattern="t\w+").fit(corpus)
vect.vocabulary_ 

In [ ]:
#tokenizer: 인수로 nltk에 있는 토크나이저 선택 가능
import nltk
vect = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
vect.vocabulary_ 

In [ ]:
#n-그램: 단어장 생성에 사용할 토큰 크기 결정.
#모노그램(1-그램)은 토큰 하나만 단어로 사용, 바이그램(2-그램)은 두 개의 연결된 토큰을 하나의 단어로 사용
#구를 토큰으로 사용하겠다 라는 의미.
vect = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
vect.vocabulary_ 

In [ ]:
#빈도 수를 이용한 stop word 이용.
#max_df, min_df 인수를 이용해, 문서에서 토큰이 나타난 횟수를 기준으로 단어장 구성 가능
#max_df를 초과하거나, min_df보다 작은 경우에는 무시. (정수는 횟수, 부동소수점은 비중)
vect = CountVectorizer(max_df=4, min_df=2).fit(corpus)
vect.vocabulary_ , vect.stop_words_

### TFidVectorizer
- Term Frequency - Inverse Document Frequency
- 단어를 갯수 그대로 카운팅X, 공통적으로 들어 있는 단어의 경우 문서별 구분 능력 적다 판단하고 가중치를 축소하는 방법
- 단어수에 idf(t)를 곱해줌.: inverse document frequency
- idf(d, t) = log n / (1 + df(t) )
- 중요도가 낮은 단어는 숫자가 더 내려감.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidv = TfidfVectorizer().fit(corpus)
tfidv.transform(corpus).toarray()

### HashingVectorizer
- Hash Function를 이용해서 문자열을 숫자로 변환시킴
- 처리할 문서가 커지면 단어장이 커지는데, hash trick 이용하면 문자열 검색 부분이 없기 때문에 시간이 유리
- 단어에 대한 인덱스 번호를 수식으로 생성
- 사전메모리 없고 실행 시간 줄일 수 있음.
- But, 충돌이 생기는 경우가 있음.

# Gensim의 문서 전처리

## Gensim 패키지 설치

In [4]:
pip install --user gensim

  Using cached gensim-3.8.3-cp37-cp37m-win_amd64.whl (24.2 MB)
  Using cached Cython-0.29.14-cp37-cp37m-win_amd64.whl (1.7 MB)
Processing c:\users\ragnarokv\appdata\local\pip\cache\wheels\56\b5\6d\86dbe4f29d4688e5163a8b8c6b740494310040286fca4dc648\smart_open-2.1.0-py3-none-any.whl
  Using cached boto3-1.14.43-py2.py3-none-any.whl (129 kB)
  Using cached botocore-1.17.43-py2.py3-none-any.whl (6.5 MB)
  Using cached s3transfer-0.3.3-py2.py3-none-any.whl (69 kB)
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Gensim의 BOW 인코딩
- 토픽 모델링을 할 때 이용
- Dictionary class 이용
 - token2id 속성으로 사전 저장
 - doc2bow 메서드로 BOW 인코딩
- TfidModel 클래스 이용하면 TF-IDF 인코딩도 가능

In [5]:
#copus 만들기
corpus = [
    'This is the first document.',
    'This is the second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?'
]

#토큰 리스트 생성
token_list = [[text for text in doc.split()] for doc in corpus]
token_list

[['This', 'is', 'the', 'first', 'document.'],
 ['This', 'is', 'the', 'second', 'document.'],
 ['And', 'the', 'third', 'one.'],
 ['Is', 'this', 'the', 'first', 'document?'],
 ['The', 'last', 'document?']]

In [6]:
#DIctionary 객체 생성
from gensim.corpora import Dictionary

dictionary = Dictionary(token_list)
dictionary.token2id

{'This': 0,
 'document.': 1,
 'first': 2,
 'is': 3,
 'the': 4,
 'second': 5,
 'And': 6,
 'one.': 7,
 'third': 8,
 'Is': 9,
 'document?': 10,
 'this': 11,
 'The': 12,
 'last': 13}

In [8]:
#BOW 인코딩
term_matrix = [dictionary.doc2bow(token) for token in token_list]
term_matrix
#각 단어가 몇번 쓰였는지 sparse matrix형태로 나온다.

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(0, 1), (1, 1), (3, 1), (4, 1), (5, 1)],
 [(4, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (4, 1), (9, 1), (10, 1), (11, 1)],
 [(10, 1), (12, 1), (13, 1)]]

In [9]:
#TF-IDF 인코딩
from gensim.models import TfidfModel

tfidf = TfidfModel(term_matrix) #카운팅 매트릭스를 그대로 넣게 되면, tfid로 바뀜.

for doc in tfidf[term_matrix]:
    print("doc:")
    for k, v in doc:
        print(k, v)

doc:
0 0.49633406058198626
1 0.49633406058198626
2 0.49633406058198626
3 0.49633406058198626
4 0.12087183801361165
doc:
0 0.4034194772828018
1 0.4034194772828018
3 0.4034194772828018
4 0.09824442362969368
5 0.7085945309359098
doc:
4 0.07979258234193365
6 0.5755093812740171
7 0.5755093812740171
8 0.5755093812740171
doc:
2 0.3485847413542797
4 0.08489056411237639
9 0.6122789185961829
10 0.3485847413542797
11 0.6122789185961829
doc:
10 0.37344696513776354
12 0.6559486886294514
13 0.6559486886294514
